In [13]:
%load_ext autoreload
%autoreload 2
import torch
from transformer_vq.nn.attn_vq import VQAttentionQK
import yaml
from einops import rearrange

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
with open('conf.yml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
n_vocab = config['n_vocab']
sequence_length = config['sequence_len']
batch_size = config['batch_size']
vocabularies = torch.randint(low=0, high=n_vocab, size=(4, sequence_length))

KeyError: 'batch_size'